In [26]:
import nbimporter
from dotenv import load_dotenv
from imagegenerator import ImageGenerator
import gradio as gr
import os
import json
from openai import OpenAI

In [27]:
# Initialization

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [28]:
imggen = ImageGenerator()
#prompt_text = "An image representing vacation in city New York having its famous tourist spots"
#imggen.artist(prompt_text)

OpenAI API Key exists


In [29]:
ticket_prices = {"new york": "$500", "london": "$1200", "tokyo" : "$1500"}

def get_ticket_price(city: str):
    print(f"Tool get_ticket_price called for city {city}")
    return ticket_prices.get(city.lower(), "Unknown")


In [30]:
#tool function description for model

tool_description = {
    "name": "get_ticket_price",
    "description": "A function to get price for ticket to destination. Call this function whenever you need to know the ticket price. for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties" : {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}



In [31]:
#list of tools to provide model

tools = [{"type": "function", "function": tool_description}]

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city
    
    

In [40]:
system_message = "You are a helpful airline assistant called Airline AI."
system_message += "Give simple courteous replies not ,more than 1 line. if you dont know the answer, say so"

def get_image_prompt(city):
    prompt_text = f"An image representing vacation in city {city} having its famous tourist spots"
    return prompt_text
    
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = imggen.artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    
    return history, image

#gr.ChatInterface(fn=chat, type="messages").launch()


In [44]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our Airline Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=[], outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for city londin


Traceback (most recent call last):
  File "/Users/mannatwadhwani/Learning_Workspace/my_llm_learning/llms/lib/python3.10/site-packages/gradio/routes.py", line 990, in predict
    output = await route_utils.call_process_api(
  File "/Users/mannatwadhwani/Learning_Workspace/my_llm_learning/llms/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/mannatwadhwani/Learning_Workspace/my_llm_learning/llms/lib/python3.10/site-packages/gradio/blocks.py", line 2057, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/Users/mannatwadhwani/Learning_Workspace/my_llm_learning/llms/lib/python3.10/site-packages/gradio/blocks.py", line 1863, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/Users/mannatwadhwani/Learning_Workspace/my_llm_learning/llms/lib/python3.10/site-packages/gradio/components/chatbot.py", line 564, in po